# 1. Library

In [24]:
import os
import torch
from torchvision import transforms
import pandas as pd
from torch.utils.data.dataset import Dataset
from sklearn.model_selection import train_test_split

import sklearn.preprocessing
from directory_tree import display_tree
# Customed Library
import engine ,model_builder,utils
import data_loader
import numpy as np
import torchinfo
from timeit import default_timer as timer 
#display_tree('./')

# 2. Data loading

In [9]:
path='./csv_files'
Desired_param=[[0,1],[0,1],[0],[0]]
A=data_loader.data_loader(path,Desired_param)
A.file_searching(path)
list_file=A.desired_file_path(Desired_param)
Columns=['Force_X','Force_Y','Force_Z','Torque_X','Torque_Y','Torque_Z',
         'Sensor_P_1_1','Sensor_P_1_2','Sensor_P_2_1','Sensor_P_2_2',
         'Sensor_P_3_1','Sensor_P_3_2','Sensor_P_4_1','Sensor_P_4_2',
         'Sensor_P_5_1','Sensor_P_5_2','Sensor_P_6_1','Sensor_P_6_2',
         'Sensor_P_7_1','Sensor_P_7_2',
         'Sensor_S']
New_Columns=['Pressure','Object_size','Finger_distance','Trial']
Total_dataset=A.read_Data_pandas(Columns,New_Columns)
Total_dataset

1st Folder name :  P0.7_L_cen_1
Contents in the folder : ['ANALOGUE.csv', 'Force.csv', 'TCP.csv'] 


1 Parameters ['P0.7' 'P1.4' 'P2.1'] 

2 Parameters ['L' 'M' 's'] 

3 Parameters ['cen'] 

4 Parameters ['1' '2' '3'] 



,Force_X,Force_Y,Force_Z,Torque_X,Torque_Y,Torque_Z,Sensor_P_1_1,Sensor_P_1_2,Sensor_P_2_1,Sensor_P_2_2,...,Sensor_P_5_2,Sensor_P_6_1,Sensor_P_6_2,Sensor_P_7_1,Sensor_P_7_2,Sensor_S,Pressure,Object_size,Finger_distance,Trial
0,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.118,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
1,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
2,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
3,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.183,10.198,5.669,14.974,P0.7,L,cen,1
4,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.183,10.198,5.669,14.974,P0.7,L,cen,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
20623,-0.003196,0.022557,0.039090,0.000388,-0.000059,-0.000344,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P1.4,M,cen,1
20624,-0.003196,0.022557,0.039090,0.000388,-0.000059,-0.000344,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P1.4,M,cen,1
20625,-0.003186,0.022070,0.037911,0.000373,-0.000106,-0.000351,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P1.4,M,cen,1
20626,-0.003186,0.022070,0.037911,0.000373,-0.000106,-0.000351,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P1.4,M,cen,1


### 3-1. FIltering

In [11]:
## Data filtering
# https://builtin.com/data-science/pandas-filter
Filtered_dataset=Total_dataset.query('Pressure=="P0.7" and Object_size=="L"')
Filtered_dataset


,Force_X,Force_Y,Force_Z,Torque_X,Torque_Y,Torque_Z,Sensor_P_1_1,Sensor_P_1_2,Sensor_P_2_1,Sensor_P_2_2,...,Sensor_P_5_2,Sensor_P_6_1,Sensor_P_6_2,Sensor_P_7_1,Sensor_P_7_2,Sensor_S,Pressure,Object_size,Finger_distance,Trial
0,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.118,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
1,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
2,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.181,10.200,5.680,14.974,P0.7,L,cen,1
3,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.183,10.198,5.669,14.974,P0.7,L,cen,1
4,-0.001032,0.003147,0.027862,0.000034,0.000008,-0.000392,0.0,0.0,3.674,7.520,...,9.124,7.565,8.183,10.198,5.669,14.974,P0.7,L,cen,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5152,-0.003196,0.022557,0.039090,0.000388,-0.000059,-0.000344,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P0.7,L,cen,1
5153,-0.003196,0.022557,0.039090,0.000388,-0.000059,-0.000344,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P0.7,L,cen,1
5154,-0.003186,0.022070,0.037911,0.000373,-0.000106,-0.000351,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P0.7,L,cen,1
5155,-0.003186,0.022070,0.037911,0.000373,-0.000106,-0.000351,0.0,0.0,3.675,7.521,...,9.126,7.563,8.174,10.192,5.672,14.974,P0.7,L,cen,1


### 3-2 Combination of the data

In [12]:
Total_col=[*Columns,*New_Columns]
for i in range(len(Total_col)):
    if i==0:
        print(f'{"Total column" :^120}')
        print('')
    print(f'{i:<} {Total_col[i] : <30}   ',end='')
    if i%3 ==0 and i!=0:
        print('')
        

Desired_combination_COL=[*Total_col[0:3],*Total_col[-9:-5]] ####################### chose the combination of the col
print('')
for i in range(len(Desired_combination_COL)):
    if i==0:
        print(f'{"Desired_combination_COL" :^120}')
        print('')
    print(f'{i:<} {Desired_combination_COL[i] : <30}   ',end='')
    if i%3 ==0 and i!=0:
        print('')


Desired_combination_COL
Combi_df=Filtered_dataset.filter(items=Desired_combination_COL)

                                                      Total column                                                      

0 Force_X                          1 Force_Y                          2 Force_Z                          3 Torque_X                         
4 Torque_Y                         5 Torque_Z                         6 Sensor_P_1_1                     
7 Sensor_P_1_2                     8 Sensor_P_2_1                     9 Sensor_P_2_2                     
10 Sensor_P_3_1                     11 Sensor_P_3_2                     12 Sensor_P_4_1                     
13 Sensor_P_4_2                     14 Sensor_P_5_1                     15 Sensor_P_5_2                     
16 Sensor_P_6_1                     17 Sensor_P_6_2                     18 Sensor_P_7_1                     
19 Sensor_P_7_2                     20 Sensor_S                         21 Pressure                         
22 Object_size                      23 Finger_distance                  24 Trial         

### 3. Save the file into pickle format

In [13]:
save_mode= True

if save_mode== True:
    Combi_df.to_pickle("./pickle/test_1.pkl") 

# 3.Start

### 3-1. Data loading and preparation
1) Read pickle
2) Divide Input and output && Train and test set
3) To Tensor
4) Normalization
5) Batch size

(tensor([0.5833, 0.7500, 0.5946, 0.3571]), tensor([0.3498, 0.5269, 0.0082]))

In [16]:
Combi_df = pd.read_pickle("./pickle/test_1.pkl")  
# Hot encoding needed
# Scailing!
# Validation dataset?

Y=np.array(Combi_df.filter(items=Desired_combination_COL[0:3]))
X=np.array(Combi_df.filter(items=Desired_combination_COL[3:]))

# Scaler
X_scaler = sklearn.preprocessing.MinMaxScaler()
Y_scaler = sklearn.preprocessing.MinMaxScaler()

X=torch.FloatTensor(X_scaler.fit_transform(X))
Y=torch.FloatTensor(Y_scaler.fit_transform(Y))



X_train, X_test, Y_train, Y_test = train_test_split(X, 
                                                    Y, 
                                                    test_size=0.2, # 20% test, 80% train
                                                    random_state=42) # make the random split reproducible


train_dataset=data_loader.Data(X_train,Y_train)
test_dataset=data_loader.Data(X_test,Y_test)
Input_dim=len(X_train[0])
Output_dim=len(Y_train[0])

print(f'{"Input dim ":>20}{Input_dim}{"||":^22}{"Output dim ":>20}{Output_dim}')
print(f'Input Train: {len(X_train)},Test: {len(X_test)}   ||   Output Train: {len(Y_train)}, Test: {len(Y_test)}')

          Input dim 4          ||                   Output dim 3
Input Train: 4125,Test: 1032   ||   Output Train: 4125, Test: 1032


### 3-2. Set hypermeter
1. Epochs
2. Batch size
3. Learning rate
4. Device 
5. Activation function
6. Loss function
7. Optimizer

In [26]:
# Setup hyperparameters
NUM_EPOCHS = 5
BATCH_SIZE = 32
HIDDEN_UNITS = 10
LEARNING_RATE = 0.001
device = "cuda" if torch.cuda.is_available() else "cpu"


# Model

model = model_builder.LSTMModel(
    input_dim = Input_dim,
    hidden_dim=HIDDEN_UNITS,
    layer_dim=5,
    output_dim=Output_dim,
    dropout_prob=0.7)


# Set loss and optimizer
loss_fn = torch.nn.CrossEntropyLoss() ## change
optimizer = torch.optim.Adam(model.parameters(), ## change
                             lr=LEARNING_RATE)


TypeError: __init__() got an unexpected keyword argument 'dropout_prob'

### 3-3. Model


In [25]:

#torchinfo.summary(model,(5,5,5))
start_time = timer()
results=engine.train(model=model,
             train_dataloader=train_dataset,
             test_dataloader=test_dataset,
             loss_fn=loss_fn,
             optimizer=optimizer,
             epochs=NUM_EPOCHS,
             device=device)
end_time = timer()
print(f"Total training time: {end_time-start_time:.3f} seconds")
utils.plot_loss_curves(results)

TypeError: __init__() got an unexpected keyword argument 'dropout_prob'

### 3-4 Compare the results and save